In [212]:
# Download file
import urllib.request
urllib.request.urlretrieve("https://data-engineer-technical-challenge.s3.ap-southeast-2.amazonaws.com/pace-data.txt", "pace-data.txt")

('pace-data.txt', <http.client.HTTPMessage at 0x7f4cb96aa590>)

In [213]:
# 2. Write a script or notebook to perform the following operations on the data 

In [214]:
# a) Extract the data from the CSV
import pandas as pd

df = pd.read_csv("pace-data.txt")
print(df.head())

      MovementDateTime Destination       DestinationTidied  Speed  \
0  2019-07-20 00:00:00    PATIMBAN                     NaN    9.8   
1  2019-07-20 00:00:00      LA PAZ                  La Paz    0.0   
2  2019-07-20 00:00:00       GTPRQ          Puerto Quetzal   13.3   
3  2019-07-20 00:00:00      RU LED  St Petersburg Ugolnaya    0.0   
4  2019-07-20 00:00:00      KAMSAR             Port Kamsar   13.6   

  AdditionalInfo CallSign  Heading       MMSI        MovementID  \
0            NaN     YDLR     90.0  525019348  4780574056319002   
1            NaN    XCAR8    163.0  345060069  4681115056319002   
2            NaN  V7A2042    118.0  538008172  4984813056319002   
3            NaN    D5SQ4    321.0  636019034  4977443056319002   
4            NaN    3FBS5    216.0  372798000  4978315056319002   

           ShipName ShipType  Beam  Draught  Length                      ETA  \
0         MV SENANG    Cargo    16      6.8     117  2019-07-20 16:00:00.000   
1           SAKIS P   

In [215]:
# b) Normalize MovementDateTime to ISO format 
import datetime

print(df['MovementDateTime'].head())

df['MovementDateTime'] = df['MovementDateTime'].map(lambda x: datetime.datetime.fromisoformat(x).isoformat())
print(df['MovementDateTime'].head())

0    2019-07-20 00:00:00
1    2019-07-20 00:00:00
2    2019-07-20 00:00:00
3    2019-07-20 00:00:00
4    2019-07-20 00:00:00
Name: MovementDateTime, dtype: object
0    2019-07-20T00:00:00
1    2019-07-20T00:00:00
2    2019-07-20T00:00:00
3    2019-07-20T00:00:00
4    2019-07-20T00:00:00
Name: MovementDateTime, dtype: object


In [216]:
# c) For each ship grouped by CallSign, if the MoveStatus is “Under way using engine”,
# fill in any missing or zero speeds with the average of all speeds for that CallSign 

pd.options.mode.chained_assignment = None  # default='warn'

### Mean value ###
CallSign_mean_value = df.groupby('CallSign').mean(numeric_only=True)
CallSign_mean_speed_value = CallSign_mean_value["Speed"]
print(CallSign_mean_speed_value.head())

### Fetch broken values ###
df_weird_vals = df[(df["MoveStatus"] == "Under way using engine") &
                    (df["Speed"] == 0 | df["Speed"].isnull()) ]
print(df_weird_vals.head())

### Fetch broken values ###
for index, row in df_weird_vals.iterrows():
    df_weird_vals["Speed"] = CallSign_mean_speed_value[row["CallSign"]]

# 105 value is 'broken
print(df_weird_vals.head())
print("\n ORIGINAL VALUE\n")
print(df.iloc[[105]])

### Update our dataframe ###
df.update(df_weird_vals)

print("\nUPDATED VALUE\n")
print(df.iloc[[105]])

CallSign
0           4.084416
0000        0.904167
000000      4.580000
0000000     3.376471
00601      11.600000
Name: Speed, dtype: float64
        MovementDateTime   Destination DestinationTidied  Speed  \
105  2019-07-20T00:03:00  BENGKULU_IND               NaN    0.0   
124  2019-07-20T00:04:00      TG.PRIOK     Tanjung Priok    0.0   
131  2019-07-20T00:04:00   SGSIN.PEBGC         Singapore    0.0   
423  2019-07-20T00:14:00        HCM VN       Saigon East    0.0   
531  2019-07-20T00:18:00      IMBITUBA          Imbituba    0.0   

    AdditionalInfo CallSign  Heading       MMSI        MovementID  \
105            NaN    DSQZ4    162.0  441763000  4948924056319002   
124            NaN     JZIG     75.0  525019643  4739702056319002   
131            NaN    DSRK3     89.0  441889000  4914980056319002   
423            NaN    3WCV9      0.0  574001010  4959979056318551   
531            NaN    V7NM2    331.0  538006478  4975409056319002   

               ShipName ShipType  Beam  

In [217]:
# d) Create a new feature called BeamRatio calculated as Beam / Length (Beam divided by Length)

df["BeamRatio"] = df["Beam"]/ df["Length"]
print(df["BeamRatio"].head())

0    0.136752
1    0.158537
2    0.160804
3    0.160000
4    0.139738
Name: BeamRatio, dtype: float64


In [218]:
# e) Save all the existing features and newly calculated features in 
# steps b, c & d as a new csv file  (enriched.csv) on your local machine. 

print(df)
df.to_csv("enriched.csv", index=False)

           MovementDateTime     Destination       DestinationTidied  Speed  \
0       2019-07-20T00:00:00        PATIMBAN                     NaN    9.8   
1       2019-07-20T00:00:00          LA PAZ                  La Paz    0.0   
2       2019-07-20T00:00:00           GTPRQ          Puerto Quetzal   13.3   
3       2019-07-20T00:00:00          RU LED  St Petersburg Ugolnaya    0.0   
4       2019-07-20T00:00:00          KAMSAR             Port Kamsar   13.6   
...                     ...             ...                     ...    ...   
174053  2019-07-20T23:59:00   VISAKHAPATNAM           Visakhapatnam    9.6   
174054  2019-07-20T23:59:00  CN TAPAN CHINA                     NaN    9.0   
174055  2019-07-20T23:59:00          KR PUS                   Busan   12.9   
174056  2019-07-20T23:59:00         NORFOLK                 Norfolk    0.0   
174057  2019-07-20T23:59:00          AU GLT               Gladstone   10.7   

       AdditionalInfo CallSign     Heading         MMSI    Move

In [219]:
# f) Use the data file enriched.csv and store it in a Postgres or MySQL table
# on your local machine (NOTE: You are expected to write python code to 
# programmatically insert the data into the database) 

# g) Provide a screen shot of the data inserted into the table 

# Wrap all your code & enriched.csv  into a private repo on Github and add  khordoo-m and cr3 as collaborators

In [220]:
# 5. Write a script (bash preferred) that uploads the pace-data.csv file into an S3 bucket (bonus 
# points for creating the bucket through the CLI). If you don’t want to write a script, write 
# down the commands you use to create the bucket and upload the file.

# IMPORTANT NOTE: The IAM role using the notebook must have the appropriate 
# authorizations. S3BucketAllAccess should be enough.

# IMPORTANT NOTE: This works when ran on AWS jupyter notebook. 
# To work im batch or command line, s3cmd must be used instead

filename="pace-data.txt"
bucket_to_upload_name = "automatically-created-bucket-ivanalt-mila"
!echo "5. Write a script (bash preferred) that uploads the pace-data.csv file into an S3 bucket (bonus points for creating the bucket through the CLI). If you don’t want to write a script, write down the commands you use to create the bucket and upload the file."
!echo ""

!aws s3 ls
!echo ""


# TODO: GRANT PERMISSIONS ?
!aws s3 mb s3://$bucket_to_upload_name

!echo "pace-data.csv must be on the same location to this script to work"
!echo ""

# This command line is slightly different in windows

!aws s3 cp $filename s3://$bucket_to_upload_name/$filename

5. Write a script (bash preferred) that uploads the pace-data.csv file into an S3 bucket (bonus points for creating the bucket through the CLI). If you don’t want to write a script, write down the commands you use to create the bucket and upload the file.

2025-04-07 18:47:29 automatically-created-bucket-ivanalt-mila
2025-03-31 19:55:34 main-6-bucket-allan
2025-04-01 20:05:28 sagemaker-build-and-deploy-youtube-example
2025-04-01 19:07:12 sagemaker-studio-211160189791-ylb8w879akd
2025-04-01 19:07:15 sagemaker-us-east-2-211160189791

make_bucket failed: s3://automatically-created-bucket-ivanalt-mila An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.
pace-data.csv must be on the same location to this script to work

upload: ./pace-data.txt to s3://automatically-created-bucket-ivanalt-mila/pace-data.txt


In [221]:
# Get the dataframe types for SQL integration later
for name, dtype in df.dtypes.iteritems():
    print(name, dtype)

MovementDateTime object
Destination object
DestinationTidied object
Speed float64
AdditionalInfo object
CallSign object
Heading float64
MMSI float64
MovementID float64
ShipName object
ShipType object
Beam float64
Draught float64
Length float64
ETA object
MoveStatus object
ladenStatus object
LRIMOShipNo float64
Latitude float64
Longitude float64
BeamRatio float64


/tmp/ipykernel_7316/16896546.py:2: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, dtype in df.dtypes.iteritems():
